### Inverse kinematics of `Thormang` with collision detection

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import mujoco_py
from mujoco_parser import *
from util import *
np.set_printoptions(precision=2)
%config InlineBackend.figure_format = 'retina'
%matplotlib inline
print ("Done.")

Done.


### Spawn `Thoramang` with objects and check collision while solving IK

In [2]:
env = MuJoCoParserClass(name='Thormang_rilab',rel_xml_path='../asset/thormang_rilab/thormang_rilab.xml',VERBOSE=True)
env.init_viewer(TERMINATE_GLFW=True,INITIALIZE_GLFW=True,window_width=0.5,window_height=0.5)
env.set_max_tick(max_tick=10000)

# Init IK
body_name = 'l_arm_wr_p_link'
q         = env.get_q_pos(q_pos_idxs=env.rev_joint_idxs)
p_EE      = env.get_p_body(body_name=body_name)
R_EE      = env.get_R_body(body_name=body_name)
p_trgt    = p_EE + np.array([-0.3,-0.3,-0.3])
R_trgt    = R_EE

# Buffers
err_list = np.zeros(env.max_tick)
q_list   = np.zeros((env.max_tick,env.n_rev_joint))
while env.IS_ALIVE():
    # Numerical IK
    dq,err = env.one_step_ik(body_name=body_name,p_trgt=p_trgt,R_trgt=R_trgt,th=3.0*np.pi/180.0)
    q = q + dq
    # FK
    env.forward(q_pos=q,q_pos_idxs=env.rev_joint_idxs)
    # Render
    env.add_marker(env.get_p_body(body_name),radius=0.02,color=np.array([0,1,0,0.5]))
    env.add_marker(p_trgt,radius=0.02,color=np.array([0,1,0,0.5])) 
    
    for c_idx in range(env.sim.data.ncon):
        contact    = env.sim.data.contact[c_idx]
        p_contact  = contact.pos
        bodyname1  = env.body_idx2name(env.sim.model.geom_bodyid[contact.geom1])
        bodyname2  = env.body_idx2name(env.sim.model.geom_bodyid[contact.geom2])
        label      = '%s-%s'%(bodyname1,bodyname2)
        env.add_marker(pos=p_contact,radius=0.1,color=np.array([1,0,0,0.5]),label=label)
    
    env.render(RENDER_ALWAYS=True)
    # Append
    q_list[env.tick-1,:] = env.get_q_pos(q_pos_idxs=env.rev_joint_idxs)
env.terminate_viewer()
print ("Done.")

[Thormang_rilab] Instantiated from [/home/user/Workspace/simple-mujoco-usage-v2/asset/thormang_rilab/thormang_rilab.xml]
- Simulation timestep is [0.0010]sec and frequency is [1000]HZ
- [Thormang_rilab] has [37] bodies
 [00] body name:[world]
 [01] body name:[pelvis_link]
 [02] body name:[chest_link]
 [03] body name:[l_arm_sh_p1_link]
 [04] body name:[l_arm_sh_r_link]
 [05] body name:[l_arm_sh_p2_link]
 [06] body name:[l_arm_el_y_link]
 [07] body name:[l_arm_wr_r_link]
 [08] body name:[l_arm_wr_y_link]
 [09] body name:[l_arm_wr_p_link]
 [10] body name:[l_arm_grip_link]
 [11] body name:[l_arm_grip_1_link]
 [12] body name:[r_arm_sh_p1_link]
 [13] body name:[r_arm_sh_r_link]
 [14] body name:[r_arm_sh_p2_link]
 [15] body name:[r_arm_el_y_link]
 [16] body name:[r_arm_wr_r_link]
 [17] body name:[r_arm_wr_y_link]
 [18] body name:[r_arm_wr_p_link]
 [19] body name:[r_arm_grip_link]
 [20] body name:[r_arm_grip_1_link]
 [21] body name:[head_y_link]
 [22] body name:[head_p_link]
 [23] body name:[l

SystemExit: 0

/home/user/.pyenv/versions/3.8.0/envs/mujoco_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
env.terminate_viewer()